In [1]:
!pip install flask
!pip install nltk
!pip install urllib3
!pip install bs4
!pip install plotly
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Importing all the libraries

In [3]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

# for extracting data from finviz
finviz_url = 'https://finviz.com/quote.ashx?t='

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saish\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Getting HTML Headlines from FinViz Using Beautiful Soup

In [4]:
def get_news(ticker):
    url = finviz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    return news_table

ticker = 'AMZN'
amz = get_news(ticker)
amz # pure html code, data is not parsed yet

<table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer" id="news-table" width="100%">
<tr><td align="right" width="130">Apr-27-23 08:06PM</td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/m/f3bcc7e4-fded-3001-8564-a2206ed1629d/amazon-stock-shocks.html" onclick="trackAndOpenNews(event, 'Motley Fool', 'https://finance.yahoo.com/m/f3bcc7e4-fded-3001-8564-a2206ed1629d/amazon-stock-shocks.html');" rel="nofollow" target="_blank">Amazon Stock Shocks Investors: What Happened?</a></div><div class="news-link-right"><span>(Motley Fool)</span></div></div></td></tr>
<tr><td align="right" width="130">07:59PM</td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/news/amazon-stock-forfeits-gains-after-company-warns-on-optimizations-from-aws-customers-235944403.html" onclick="trackAndOpenNews(event, 'Yahoo Fin

## Parsing News into DF

In [22]:
def parse_news(news_table):
    parsed_news = []
    
    for x in news_table.findAll('tr'):
        # check if 'a' tag is present in the 'tr' tag
        if x.a is not None:
            # read the text from 'a' tag into text
            text = x.a.get_text() 
            # splite text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element
            if len(date_scrape) == 1:
                time = date_scrape[0]
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            # Append ticker, date, time and headline as a list to the 'parsed_news' list
            parsed_news.append([date, time, text])        
    # Set column names
    columns = ['date', 'time', 'headline']
    # Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
    parsed_news_df = pd.DataFrame(parsed_news, columns=columns)        
    # Create a pandas datetime object from the strings in 'date' and 'time' column
    parsed_news_df['datetime'] = pd.to_datetime(parsed_news_df['date'] + ' ' + parsed_news_df['time'])
    
    return parsed_news_df


In [23]:
parsed_news_df = parse_news(amz)


In [25]:
parsed_news_df

,date,time,headline,datetime
0,Apr-27-23,08:06PM,Amazon Stock Shocks Investors: What Happened?,2023-04-27 20:06:00
1,Apr-27-23,07:59PM,Amazon stock forfeits gains after company warn...,2023-04-27 19:59:00
2,Apr-27-23,07:53PM,Amazon says cloud growth slowed as customers c...,2023-04-27 19:53:00
3,Apr-27-23,07:43PM,Amazon Jolts Investors With Talk of Cloud Grow...,2023-04-27 19:43:00
4,Apr-27-23,07:35PM,Amazon's Q1 earnings call: What Wall Street is...,2023-04-27 19:35:00
...,...,...,...,...
95,Apr-25-23,05:00AM,"Zacks Industry Outlook Highlights CVS Health, ...",2023-04-25 05:00:00
96,Apr-25-23,05:00AM,"Alphabet, Microsoft, Meta Platforms, Amazon an...",2023-04-25 05:00:00
97,Apr-24-23,09:23PM,Time to Buy Amazon or Microsoft Stock with Ear...,2023-04-24 21:23:00
98,Apr-24-23,06:59PM,Amazon Faces Backlash Over One Of Its Budget Cuts,2023-04-24 18:59:00


## Scores for Sentiment Analysis

In [26]:
def score_news(parsed_news_df):
    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    
    # Iterate through the headlines and get the polarity scores using vader
    scores = parsed_news_df['headline'].apply(vader.polarity_scores).tolist()

    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)

    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = parsed_news_df.join(scores_df, rsuffix='_right')        
    parsed_and_scored_news = parsed_and_scored_news.set_index('datetime')    
    parsed_and_scored_news = parsed_and_scored_news.drop(['date', 'time'], 1)          
    parsed_and_scored_news = parsed_and_scored_news.rename(columns={"compound": "sentiment_score"})

    return parsed_and_scored_news

parsed_and_scored_news = score_news(parsed_news_df)
parsed_and_scored_news.head()

C:\Users\saish\AppData\Local\Temp\ipykernel_31392\109050321.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  parsed_and_scored_news = parsed_and_scored_news.drop(['date', 'time'], 1)


,headline,neg,neu,pos,sentiment_score
datetime,,,,,
2023-04-27 20:06:00,Amazon Stock Shocks Investors: What Happened?,0.313,0.482,0.205,-0.2263
2023-04-27 19:59:00,Amazon stock forfeits gains after company warn...,0.097,0.621,0.283,0.4019
2023-04-27 19:53:00,Amazon says cloud growth slowed as customers c...,0.169,0.484,0.347,0.2960
2023-04-27 19:43:00,Amazon Jolts Investors With Talk of Cloud Grow...,0.000,0.619,0.381,0.5106
2023-04-27 19:35:00,Amazon's Q1 earnings call: What Wall Street is...,0.000,1.000,0.000,0.0000


## Hourly Sentiment 

In [27]:
def plot_hourly_sentiment(parsed_and_scored_news, ticker):
   
    # Group by date and ticker columns from scored_news and calculate the mean
    mean_scores = parsed_and_scored_news.resample('H').mean()

    # Plot a bar chart with plotly 
    fig = px.bar(mean_scores, x=mean_scores.index, y='sentiment_score', title = ticker + ' Hourly Sentiment Scores')
    fig.show()
    
plot_hourly_sentiment(parsed_and_scored_news, ticker)

C:\Users\saish\AppData\Local\Temp\ipykernel_31392\3066297716.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_scores = parsed_and_scored_news.resample('H').mean()


## Daily Sentiment

In [39]:
def plot_daily_sentiment(parsed_and_scored_news, ticker):
   
    # Group by date and ticker columns from scored_news and calculate the mean
    mean_scores = parsed_and_scored_news.resample('D').mean()

    # Plot a bar chart with plotly
    fig = px.bar(mean_scores, x=mean_scores.index, y='sentiment_score', title = ticker + ' Daily Sentiment Scores', color_discrete_sequence = ["coral"])
    fig.update_xaxes(title_text='Date and Time')
    fig.update_yaxes(title_text='Sentiment Score')
    fig.show()
    
plot_daily_sentiment(parsed_and_scored_news, ticker)

C:\Users\saish\AppData\Local\Temp\ipykernel_31392\2712842648.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [35]:
import numpy as np

def plot_hourly_sentiment(parsed_and_scored_news, ticker):
    # Group by date and ticker columns from parsed_and_scored_news and calculate the mean
    mean_scores = parsed_and_scored_news.resample('H').mean()
    
    # Create a new column called 'color' that maps positive sentiment scores to green and negative scores to red
    mean_scores['color'] = np.where(mean_scores['sentiment_score'] > 0, 'green', 'red')

    # Plot a bar chart with plotly, setting the color of each bar based on the 'color' column
    fig = px.bar(mean_scores, x=mean_scores.index, y='sentiment_score', title=ticker + ' Hourly Sentiment Scores',
             color='color', color_discrete_map={'green': 'green', 'red': 'red'})
    # Update the x and y axis labels
    fig.update_xaxes(title_text='Date and Time')
    fig.update_yaxes(title_text='Sentiment Score')

    # Add custom labels for positive and negative sentiment scores
    fig.update_layout(legend=dict(title='', orientation='h', y=1.1, x=0.5, xanchor='center'),
                    coloraxis_colorbar=dict(title='Sentiment', tickvals=[-1, 0, 1],
                                            ticktext=['Negative', 'Neutral', 'Positive'],
                                            tickmode='array', ticks='outside'))

    # Remove the legend
    fig.update_layout(showlegend=False)

    fig.show()


plot_hourly_sentiment(parsed_and_scored_news, ticker)


C:\Users\saish\AppData\Local\Temp\ipykernel_31392\474792285.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

